# Benchmarking Defesa - Dataset LIAA - Aprendizado Raso em Atributos RQA

Estruturação de pipeline baseado em aprendizado raso utilizando atributos de alta frequência.


# Configurações

In [1]:
import os
import sys
import gc
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from tqdm import *

from pretty_confusion_matrix import *

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS
RESIDENCIA = 3

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "28") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

In [3]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

# Carregando Dados

In [4]:
# Carregando datasets
df_treino = pd.read_csv(os.path.join(caminho_dados_notebook, 'training_windows.txt'))
df_validacao = pd.read_csv(os.path.join(caminho_dados_notebook, 'validation_windows.txt'))

# Selecionando feature dominio do tempo e frequencia / outputs (status dos aparelhos - dummy)
colunas_janela = df_treino.columns[:512]

colunas_output = ['LC', 'LI', 'MO', 'MT', 'PC', 'LF']

# Preparando dados de treino e validacao
X_treino = df_treino[colunas_janela]
X_validacao = df_validacao[colunas_janela]

y_treino = df_treino[colunas_output].replace(-1, 0)
y_validacao = df_validacao[colunas_output].replace(-1, 0)

del df_treino
del df_validacao
gc.collect()

3

# Treinamento e Avaliação dos Modelos

In [5]:
# from PyNILM.dados.janelas import Janelas
from PyNILM.dados.janelas import Janelas
from PyNILM.dados.utils import *

from PyNILM.avaliacao.metricas import *
from PyNILM.avaliacao.graficos import *
from PyNILM.avaliacao.analises import *

from PyNILM.modelos.utils import *
from PyNILM.modelos.dlafe import DLAFE
from PyNILM.modelos.rqa import RQA

# Inicializar uso GPU
start_tf_session(memory_limit=int(1024*4))

Virtual devices cannot be modified after being initialized


In [6]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

## Atributos RQA

In [10]:
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation

def converter_janelas_para_rqa(
    X,
    atributos_rqa_estudo=None,
    arquivo_rqa=None):
    atributos_rqa = [
        "Minimum diagonal line length (L_min)",
        "Minimum vertical line length (V_min)",
        "Minimum white vertical line length (W_min)",
        "Recurrence rate (RR)",
        "Determinism (DET)",
        "Average diagonal line length (L)",
        "Longest diagonal line length (L_max)",
        "Divergence (DIV)",
        "Entropy diagonal lines (L_entr)",
        "Laminarity (LAM)",
        "Trapping time (TT)",
        "Longest vertical line length (V_max)",
        "Entropy vertical lines (V_entr)",
        "Average white vertical line length (W)",
        "Longest white vertical line length (W_max)",
        "Longest white vertical line length inverse (W_div)",
        "Entropy white vertical lines (W_entr)",
        "Ratio determinism / recurrence rate (DET/RR)",
        "Ratio laminarity / determinism (LAM/DET)"
    ]

    rqa_data = []
    for x in tqdm(X, total=X.shape[0]) :

        # Calculating RQA attributes
        time_series = TimeSeries(x,
                    embedding_dimension=PARAMETROS_RP["dimension"],
                    time_delay=PARAMETROS_RP["time_delay"])
        settings = Settings(time_series,
                            analysis_type=Classic,
                            neighbourhood=FixedRadius(PARAMETROS_RP["percentage"]/100), 
                            # PS.: Utilizando percentage ao inves de threshold 
                            # devido a semanticas distintas entre libs (pyts e pyrqa)
                            # bem como distincao entre RPs (cnn) e RQAs (supervisionado).
                            similarity_measure=EuclideanMetric)
        computation = RQAComputation.create(settings, verbose=False)
        rqa_result = computation.run()

        rqa_data.append( list(np.nan_to_num(rqa_result.to_array())) )

    # Numpy to Pandas 
    df_rqa = pd.DataFrame(
        data=rqa_data,
        columns=atributos_rqa
    )
    if arquivo_rqa:
        if os.path.isfile(arquivo_rqa): os.remove(arquivo_rqa)
        df_rqa.to_excel(arquivo_rqa, index=False)

    if not atributos_rqa_estudo:
        return df_rqa
    else:
        return df_rqa[atributos_rqa_estudo]

# Parametros execucao do experimento
atributos = "rqa" 

atributos_rqa_estudo = [
        "Recurrence rate (RR)",
        "Determinism (DET)"
    ]

# Carregando dados RQA (treino)
arquivo_rqa_treino = os.path.join(caminho_dados_notebook, f"rqa_treino.xlsx")
if os.path.isfile(arquivo_rqa_treino):
    X_treino = pd.read_excel(arquivo_rqa_treino,engine='openpyxl')[atributos_rqa_estudo]
else:
    X_treino = converter_janelas_para_rqa(
        X_treino.values,
        atributos_rqa_estudo=atributos_rqa_estudo,
        arquivo_rqa=arquivo_rqa_treino)

# Carregando dados RQA (validacao)
arquivo_rqa_validacao = os.path.join(caminho_dados_notebook, f"rqa_validacao.xlsx")
if os.path.isfile(arquivo_rqa_validacao):
    X_validacao = pd.read_excel(arquivo_rqa_validacao,engine='openpyxl')[atributos_rqa_estudo]
else:
    X_validacao = converter_janelas_para_rqa(
        X_validacao.values,
        atributos_rqa_estudo=atributos_rqa_estudo,
        arquivo_rqa=arquivo_rqa_validacao)


# Dados agregados (validacao cruzada)
X_cv = pd.concat([X_treino, X_validacao]).reset_index(drop=True) 
y_cv = pd.concat([y_treino, y_validacao]).reset_index(drop=True) 

### SVM

In [12]:
modelo = SVC(kernel='rbf', random_state=SEED)
nome_modelo = "svm"

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
        
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()
    
# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_{nome_modelo}.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.55      0.36      0.43     12400
           1       0.54      0.72      0.61     12800

    accuracy                           0.54     25200
   macro avg       0.54      0.54      0.52     25200
weighted avg       0.54      0.54      0.52     25200

        = Confusion Matrix:

[[4414 7986]
 [3600 9200]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.53      0.32      0.40     31000
           1       0.52      0.72      0.61     32000

    accuracy                           0.52     63000
   macro avg       0.52      0.52      0.50     63000
weighted avg       0.52      0.52      0.50     63000

        = Confusion Matrix:

[[10003 20997]
 [ 9000 23000]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.86      0.85     12400
           1       0.87      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     31000
           1       0.87      0.85      0.86     32000

    accuracy                           0.86     63000
   macro avg       0.86      0.86      0.86     63000
weighted avg       0.86      0.86      0.86     63000

        = Confusion Matrix:

[[26959  4041]
 [ 4771 27229]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.55      0.68      0.61     12400
           1       0.60      0

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.74      0.74      0.74     31000
           1       0.75      0.74      0.75     32000

    accuracy                           0.74     63000
   macro avg       0.74      0.74      0.74     63000
weighted avg       0.74      0.74      0.74     63000

        = Confusion Matrix:

[[22999  8001]
 [ 8207 23793]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.52      0.65      0.58     12400
           1       0.56      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.52      0.65      0.58     31000
           1       0.56      0.43      0.49     32000

    accuracy                           0.54     63000
   macro avg       0.54      0.54      0.53     63000
weighted avg       0.54      0.54      0.53     63000

        = Confusion Matrix:

[[20033 10967]
 [18162 13838]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                      f1  \
                            mean       std       max       min      mean   
appliance base                                                             
LC        cv            0.523857  0.003016  0.526349  0.518968  0.502690   
          treino-teste  0.540238       NaN  0.540238  0.540238  0.523034   
LF        cv            0.537635  0.005499  0.543175  0.529524  0.533119   
          treino-teste  0.536230       NaN  0.536230  0.536230  0.531654   
LI        cv            0.860127  0.003047  0.865238  0.857619  0.860123   
          treino-teste  0.854603       NaN  0.854603  0.854603  0.854601   
MO        cv            0.586937  0.003658  0.591190  0.582619  0.581805   
          treino-teste  0.571429       NaN  0.571429  0.571429  0.567506   
MT        cv            0.843175  0.005706  0.849762  0.836190  0.843059   
          treino-teste  0.828214       NaN  0.828214  0.828214  0.828063   
PC        cv            0.742730  0.003953  0.746111  0.736032  0.742689   
          treino-teste  0.734643       NaN  0.734643  0.734643  0.734638   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.002701  0.504807  0.498104  0.520714  0.002982   
          treino-teste       NaN  0.523034  0.523034  0.537359       NaN   
LF        cv            0.005471  0.538765  0.525246  0.539332  0.005515   
          treino-teste       NaN  0.531654  0.531654  0.537935       NaN   
LI        cv            0.003045  0.865232  0.857619  0.860276  0.003005   
          treino-teste       NaN  0.854601  0.854601  0.854758       NaN   
MO        cv            0.003703  0.586307  0.577364  0.588843  0.003660   
          treino-teste       NaN  0.567506  0.567506  0.573085       NaN   
MT        cv            0.005770  0.849747  0.836001  0.843794  0.005576   
          treino-teste       NaN  0.828063  0.828063  0.828894       NaN   
PC        cv            0.003962  0.746077  0.735973  0.742717  0.003974   
          treino-teste       NaN  0.734638  0.734638  0.734757       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            0.523170  0.515862  
          treino-teste  0.537359  0.537359  
LF        cv            0.544864  0.531164  
          treino-teste  0.537935  0.537935  
LI        cv            0.865345  0.857848  
          treino-teste  0.854758  0.854758  
MO        cv            0.593065  0.584536  
          treino-teste  0.573085  0.573085  
MT        cv            0.850136  0.836941  
          treino-teste  0.828894  0.828894  
PC        cv            0.746114  0.735980  
          treino-teste  0.734757  0.734757

### XGBOOST

In [14]:
modelo = XGBClassifier(eval_metric='error', random_state=SEED, n_jobs=4)
nome_modelo = "xgboost"

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
        
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()
    
# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_{nome_modelo}.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     12400
           1       0.96      0.97      0.96     12800

    accuracy                           0.96     25200
   macro avg       0.96      0.96      0.96     25200
weighted avg       0.96      0.96      0.96     25200

        = Confusion Matrix:

[[11839   561]
 [  367 12433]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[30941    59]
 [   46 31954]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     12400
           1       0.99      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[30992     8]
 [   14 31986]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     12400
           1       0.99      0

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[30975    25]
 [   30 31970]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     12400
           1       0.99      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[30987    13]
 [    8 31992]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                      f1  \
                            mean       std       max       min      mean   
appliance base                                                             
LC        cv            0.998333  0.000112  0.998413  0.998175  0.998333   
          treino-teste  0.963175       NaN  0.963175  0.963175  0.963154   
LF        cv            0.999667  0.000220  0.999921  0.999365  0.999667   
          treino-teste  0.982341       NaN  0.982341  0.982341  0.982341   
LI        cv            0.999651  0.000191  0.999841  0.999444  0.999651   
          treino-teste  0.983413       NaN  0.983413  0.983413  0.983411   
MO        cv            0.999063  0.000198  0.999206  0.998730  0.999063   
          treino-teste  0.987659       NaN  0.987659  0.987659  0.987657   
MT        cv            0.999968  0.000043  1.000000  0.999921  0.999968   
          treino-teste  0.996587       NaN  0.996587  0.996587  0.996586   
PC        cv            0.999127  0.000589  0.999524  0.998095  0.999127   
          treino-teste  0.981349       NaN  0.981349  0.981349  0.981344   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.000112  0.998412  0.998174  0.998330  0.000111   
          treino-teste       NaN  0.963154  0.963154  0.963043       NaN   
LF        cv            0.000220  0.999921  0.999365  0.999665  0.000219   
          treino-teste       NaN  0.982341  0.982341  0.982518       NaN   
LI        cv            0.000191  0.999841  0.999444  0.999652  0.000189   
          treino-teste       NaN  0.983411  0.983411  0.983507       NaN   
MO        cv            0.000198  0.999206  0.998730  0.999064  0.000198   
          treino-teste       NaN  0.987657  0.987657  0.987748       NaN   
MT        cv            0.000043  1.000000  0.999921  0.999969  0.000043   
          treino-teste       NaN  0.996586  0.996586  0.996574       NaN   
PC        cv            0.000589  0.999524  0.998095  0.999128  0.000589   
          treino-teste       NaN  0.981344  0.981344  0.981329       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            0.998410  0.998170  
          treino-teste  0.963043  0.963043  
LF        cv            0.999919  0.999370  
          treino-teste  0.982518  0.982518  
LI        cv            0.999841  0.999448  
          treino-teste  0.983507  0.983507  
MO        cv            0.999209  0.998730  
          treino-teste  0.987748  0.987748  
MT        cv            1.000000  0.999922  
          treino-teste  0.996574  0.996574  
PC        cv            0.999526  0.998095  
          treino-teste  0.981329  0.981329

### MLP

In [15]:
modelo = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(10,), random_state=SEED)
nome_modelo = "mlp"

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
        
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()
    
# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_{nome_modelo}.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     12400
           1       0.51      1.00      0.67     12800

    accuracy                           0.51     25200
   macro avg       0.25      0.50      0.34     25200
weighted avg       0.26      0.51      0.34     25200

        = Confusion Matrix:

[[    0 12400]
 [    0 12800]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.55      0.02      0.03     31000
           1       0.51      0.99      0.67     32000

    accuracy                           0.51     63000
   macro avg       0.53      0.50      0.35     63000
weighted avg       0.53      0.51      0.36     63000

        = Confusion Matrix:

[[  489 30511]
 [  393 31607]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     12400
           1       0.87      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     31000
           1       0.87      0.85      0.86     32000

    accuracy                           0.86     63000
   macro avg       0.86      0.86      0.86     63000
weighted avg       0.86      0.86      0.86     63000

        = Confusion Matrix:

[[27000  4000]
 [ 4842 27158]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.69      0.71      0.70     12400
           1       0.71      0

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.62      0.74      0.67     31000
           1       0.69      0.57      0.62     32000

    accuracy                           0.65     63000
   macro avg       0.66      0.65      0.65     63000
weighted avg       0.66      0.65      0.65     63000

        = Confusion Matrix:

[[22838  8162]
 [13882 18118]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.59      0.58      0.58     12400
           1       0.60      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.57      0.60      0.59     31000
           1       0.59      0.57      0.58     32000

    accuracy                           0.58     63000
   macro avg       0.58      0.58      0.58     63000
weighted avg       0.58      0.58      0.58     63000

        = Confusion Matrix:

[[18583 12417]
 [13807 18193]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                      f1  \
                            mean       std       max       min      mean   
appliance base                                                             
LC        cv            0.509460  0.003407  0.515556  0.507937  0.349593   
          treino-teste  0.507937       NaN  0.507937  0.507937  0.336842   
LF        cv            0.583746  0.020376  0.617698  0.566905  0.583477   
          treino-teste  0.592183       NaN  0.592183  0.592183  0.592029   
LI        cv            0.859651  0.004081  0.864762  0.854444  0.859647   
          treino-teste  0.859563       NaN  0.859563  0.859563  0.859563   
MO        cv            0.698635  0.007372  0.706349  0.690476  0.698399   
          treino-teste  0.697698       NaN  0.697698  0.697698  0.697698   
MT        cv            0.846349  0.005069  0.854524  0.842302  0.846091   
          treino-teste  0.834921       NaN  0.834921  0.834921  0.834438   
PC        cv            0.650095  0.003488  0.653810  0.644841  0.648113   
          treino-teste  0.663571       NaN  0.663571  0.663571  0.663022   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.028512  0.400597  0.336842  0.501746  0.003905   
          treino-teste       NaN  0.336842  0.336842  0.500000       NaN   
LF        cv            0.020399  0.617471  0.566580  0.583991  0.020187   
          treino-teste       NaN  0.592029  0.592029  0.592024       NaN   
LI        cv            0.004078  0.864758  0.854444  0.859828  0.004021   
          treino-teste       NaN  0.859563  0.859563  0.859742       NaN   
MO        cv            0.007200  0.705930  0.690464  0.698419  0.007164   
          treino-teste       NaN  0.697698  0.697698  0.697869       NaN   
MT        cv            0.005168  0.854429  0.841970  0.847198  0.004936   
          treino-teste       NaN  0.834438  0.834438  0.835988       NaN   
PC        cv            0.003284  0.651642  0.643282  0.651449  0.003560   
          treino-teste       NaN  0.663022  0.663022  0.664380       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            0.508732  0.500000  
          treino-teste  0.500000  0.500000  
LF        cv            0.617467  0.567482  
          treino-teste  0.592024  0.592024  
LI        cv            0.864894  0.854682  
          treino-teste  0.859742  0.859742  
MO        cv            0.705927  0.690549  
          treino-teste  0.697869  0.697869  
MT        cv            0.855144  0.843241  
          treino-teste  0.835988  0.835988  
PC        cv            0.655227  0.646053  
          treino-teste  0.664380  0.664380

### Análise dos Resultados (1)

In [18]:
df_resultados_svm = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_svm.xlsx"), engine='openpyxl')
df_resultados_svm["model"] = "SVM"

df_resultados_xgboost = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_xgboost.xlsx"), engine='openpyxl')
df_resultados_xgboost["model"] = "XGBOOST"

df_resultados_mlp = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_resultados_mlp.xlsx"), engine='openpyxl')
df_resultados_mlp["model"] = "MLP"

# df_resultados_elm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_elm.xlsx"))
# df_resultados_elm["model"] = "ELM"

df_analise = pd.concat([
    df_resultados_svm,
    df_resultados_xgboost,
    df_resultados_mlp, 
#     df_resultados_elm,  
])

print("* Análise por modelo:")
df_analise_modelo = df_analise.groupby(["model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("model")
display(df_analise_modelo)
df_analise_modelo.to_excel(os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_analise_modelos.xlsx"))

print()
print("* Análise por aparelho/modelo:")
df_analise_aparelho = df_analise.groupby(["appliance","model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "resultados", f"{atributos}_analise_aparelhos.xls"))

* Análise por modelo:


base       acc                                      f1  \
                           mean       std       max       min      mean   
model                                                                     
XGBOOST            cv  0.999302  0.000603  1.000000  0.998095  0.999301   
XGBOOST  treino-teste  0.982421  0.010961  0.996587  0.963175  0.982416   
SVM                cv  0.682410  0.141646  0.865238  0.518968  0.677247   
SVM      treino-teste  0.677560  0.146557  0.854603  0.536230  0.673249   
MLP                cv  0.691323  0.130777  0.864762  0.507937  0.664220   
MLP      treino-teste  0.692646  0.136517  0.859563  0.507937  0.663932   

                                            auc                                
              std       max       min      mean       std       max       min  
model                                                                          
XGBOOST  0.000604  1.000000  0.998095  0.999301  0.000604  1.000000  0.998095  
XGBOOST  0.010968  0.996586  0.963154  0.982453  0.011014  0.996574  0.963043  
SVM      0.147191  0.865232  0.498104  0.682613  0.141894  0.865345  0.515862  
SVM      0.151275  0.854601  0.523034  0.677798  0.146723  0.854758  0.537359  
MLP      0.175810  0.864758  0.336842  0.690439  0.132748  0.864894  0.500000  
MLP      0.190075  0.859563  0.336842  0.691667  0.138943  0.859742  0.500000


* Análise por aparelho/modelo:


acc                                \
                                    mean       std       max       min   
appliance model   base                                                   
LC        MLP     cv            0.509460  0.003407  0.515556  0.507937   
                  treino-teste  0.507937       NaN  0.507937  0.507937   
          SVM     cv            0.523857  0.003016  0.526349  0.518968   
                  treino-teste  0.540238       NaN  0.540238  0.540238   
          XGBOOST cv            0.998333  0.000112  0.998413  0.998175   
                  treino-teste  0.963175       NaN  0.963175  0.963175   
LF        MLP     cv            0.583746  0.020376  0.617698  0.566905   
                  treino-teste  0.592183       NaN  0.592183  0.592183   
          SVM     cv            0.537635  0.005499  0.543175  0.529524   
                  treino-teste  0.536230       NaN  0.536230  0.536230   
          XGBOOST cv            0.999667  0.000220  0.999921  0.999365   
                  treino-teste  0.982341       NaN  0.982341  0.982341   
LI        MLP     cv            0.859651  0.004081  0.864762  0.854444   
                  treino-teste  0.859563       NaN  0.859563  0.859563   
          SVM     cv            0.860127  0.003047  0.865238  0.857619   
                  treino-teste  0.854603       NaN  0.854603  0.854603   
          XGBOOST cv            0.999651  0.000191  0.999841  0.999444   
                  treino-teste  0.983413       NaN  0.983413  0.983413   
MO        MLP     cv            0.698635  0.007372  0.706349  0.690476   
                  treino-teste  0.697698       NaN  0.697698  0.697698   
          SVM     cv            0.586937  0.003658  0.591190  0.582619   
                  treino-teste  0.571429       NaN  0.571429  0.571429   
          XGBOOST cv            0.999063  0.000198  0.999206  0.998730   
                  treino-teste  0.987659       NaN  0.987659  0.987659   
MT        MLP     cv            0.846349  0.005069  0.854524  0.842302   
                  treino-teste  0.834921       NaN  0.834921  0.834921   
          SVM     cv            0.843175  0.005706  0.849762  0.836190   
                  treino-teste  0.828214       NaN  0.828214  0.828214   
          XGBOOST cv            0.999968  0.000043  1.000000  0.999921   
                  treino-teste  0.996587       NaN  0.996587  0.996587   
PC        MLP     cv            0.650095  0.003488  0.653810  0.644841   
                  treino-teste  0.663571       NaN  0.663571  0.663571   
          SVM     cv            0.742730  0.003953  0.746111  0.736032   
                  treino-teste  0.734643       NaN  0.734643  0.734643   
          XGBOOST cv            0.999127  0.000589  0.999524  0.998095   
                  treino-teste  0.981349       NaN  0.981349  0.981349   

                                      f1                                \
                                    mean       std       max       min   
appliance model   base                                                   
LC        MLP     cv            0.349593  0.028512  0.400597  0.336842   
                  treino-teste  0.336842       NaN  0.336842  0.336842   
          SVM     cv            0.502690  0.002701  0.504807  0.498104   
                  treino-teste  0.523034       NaN  0.523034  0.523034   
          XGBOOST cv            0.998333  0.000112  0.998412  0.998174   
                  treino-teste  0.963154       NaN  0.963154  0.963154   
LF        MLP     cv            0.583477  0.020399  0.617471  0.566580   
                  treino-teste  0.592029       NaN  0.592029  0.592029   
          SVM     cv            0.533119  0.005471  0.538765  0.525246   
                  treino-teste  0.531654       NaN  0.531654  0.531654   
          XGBOOST cv            0.999667  0.000220  0.999921  0.999365   
                  treino-teste  0.982341       NaN  0.982341  0.982341   
LI        MLP     cv            0.859647

## Atributos Domínio da Frequência

In [50]:
dominio = 'frequencia'

X_treino = X_treino_frequencia
X_validacao = X_validacao_frequencia

# Dados agregados (validacao cruzada)
X_cv = pd.concat([X_treino, X_validacao]).reset_index(drop=True) 
y_cv = pd.concat([y_treino, y_validacao]).reset_index(drop=True) 

### SVM

In [51]:
modelo = SVC(kernel='rbf', random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
    
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()
    
# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, f"resultados_{dominio}_svm.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1.00      1.00     12800

    accuracy                           1.00     25200
   macro avg       1.00      1.00      1.00     25200
weighted avg       1.00      1.00      1.00     25200

        = Confusion Matrix:

[[12400     0]
 [    0 12800]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.89      0.98      0.93     31000
           1       0.98      0.88      0.93     32000

    accuracy                           0.93     63000
   macro avg       0.93      0.93      0.93     63000
weighted avg       0.93      0.93      0.93     63000

        = Confusion Matrix:

[[30291   709]
 [ 3786 28214]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.74      0.74      0.74     12400
           1       0.75      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.74      0.72      0.73     31000
           1       0.74      0.76      0.75     32000

    accuracy                           0.74     63000
   macro avg       0.74      0.74      0.74     63000
weighted avg       0.74      0.74      0.74     63000

        = Confusion Matrix:

[[22299  8701]
 [ 7634 24366]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                     f1  \
                            mean      std       max       min      mean   
appliance base                                                            
LC        cv            1.000000  0.00000  1.000000  1.000000  1.000000   
          treino-teste  1.000000      NaN  1.000000  1.000000  1.000000   
LF        cv            0.740714  0.00331  0.744127  0.736190  0.740423   
          treino-teste  0.746032      NaN  0.746032  0.746032  0.745968   
LI        cv            1.000000  0.00000  1.000000  1.000000  1.000000   
          treino-teste  1.000000      NaN  1.000000  1.000000  1.000000   
MO        cv            1.000000  0.00000  1.000000  1.000000  1.000000   
          treino-teste  1.000000      NaN  1.000000  1.000000  1.000000   
MT        cv            1.000000  0.00000  1.000000  1.000000  1.000000   
          treino-teste  1.000000      NaN  1.000000  1.000000  1.000000   
PC        cv            0.928651  0.00595  0.935397  0.923413  0.928570   
          treino-teste  0.871310      NaN  0.871310  0.871310  0.870996   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
LF        cv            0.003435  0.743966  0.735730  0.740380  0.003423   
          treino-teste       NaN  0.745968  0.745968  0.745968       NaN   
LI        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
MO        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
MT        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
PC        cv            0.005976  0.935340  0.923316  0.929408  0.005882   
          treino-teste       NaN  0.870996  0.870996  0.872312       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
LF        cv            0.743916  0.735713  
          treino-teste  0.745968  0.745968  
LI        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
MO        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
MT        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
PC        cv            0.936104  0.924209  
          treino-teste  0.872312  0.872312

### XGBOOST

In [52]:
modelo = XGBClassifier(eval_metric='error', random_state=SEED, n_jobs=4)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
    
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()

# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, f"resultados_{dominio}_xgboost.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     12400
           1       1.00      0.98      0.99     12800

    accuracy                           0.99     25200
   macro avg       0.99      0.99      0.99     25200
weighted avg       0.99      0.99      0.99     25200

        = Confusion Matrix:

[[12381    19]
 [  239 12561]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[30999     1]
 [    1 31999]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     12400
           1       0.98      1

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     12400
           1       0.97      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    2 31998]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                      f1  \
                            mean       std       max       min      mean   
appliance base                                                             
LC        cv            0.999968  0.000043  1.000000  0.999921  0.999968   
          treino-teste  0.989762       NaN  0.989762  0.989762  0.989761   
LF        cv            0.999968  0.000071  1.000000  0.999841  0.999968   
          treino-teste  0.960238       NaN  0.960238  0.960238  0.960235   
LI        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
MO        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  0.987302       NaN  0.987302  0.987302  0.987292   
MT        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
PC        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.000043  1.000000  0.999921  0.999968  0.000043   
          treino-teste       NaN  0.989761  0.989761  0.989898       NaN   
LF        cv            0.000071  1.000000  0.999841  0.999969  0.000070   
          treino-teste       NaN  0.960235  0.960235  0.960330       NaN   
LI        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
MO        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  0.987292  0.987292  0.987113       NaN   
MT        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
PC        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            1.000000  0.999919  
          treino-teste  0.989898  0.989898  
LF        cv            1.000000  0.999844  
          treino-teste  0.960330  0.960330  
LI        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
MO        cv            1.000000  1.000000  
          treino-teste  0.987113  0.987113  
MT        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
PC        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000

### MLP

In [53]:
modelo = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(10,), random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": []
}

for rotulo_aparelho in colunas_output:
    
    print("****************************************************************\n")
    print(f"* Aparelho `{rotulo_aparelho}`...\n")
    
    #######################################################################
    #                AVALIACAO 1 - Base de treino/validacao               #
    #######################################################################

    # # Filtrando output/status por aparelho
    y_treino_aparelho = y_treino[rotulo_aparelho]
    y_validacao_aparelho = y_validacao[rotulo_aparelho]

    print(f"  - Avaliando modelo através da base treino/validacao...")
    
    print("     -> Detalhes da amostragem (lote validacao):")
    print("     ---")
    for item in Counter(y_validacao_aparelho).items():
        print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_validacao_aparelho)*100,1)}%)" )
    print()

    # Treinando modelo
    print(f"     -> Treinando modelo...\n")
    clf = clone(modelo)
    
    clf.fit(X_treino, y_treino_aparelho)

    # Prevendo conjunto de dados
    y_hat = clf.predict(X_validacao)

    # Incrementando resultados
    resultados_modelo["appliance"].append(rotulo_aparelho)
    resultados_modelo["fold"].append("-")
    resultados_modelo["acc"].append( accuracy_score(y_validacao_aparelho, y_hat) )
    resultados_modelo["f1"].append( f1_score(y_validacao_aparelho, y_hat, average="macro") )
    resultados_modelo["auc"].append(roc_auc_score(y_validacao_aparelho, y_hat) if np.unique(y_validacao_aparelho).shape[0]>1 else 0.5)
    resultados_modelo["base"].append("treino-teste")

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_validacao_aparelho, y_hat))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_validacao_aparelho, y_hat))
    print()
    
    
    #######################################################################
    #                  AVALIACAO 2 - Validacao Cruzada                    #
    #######################################################################
    
    y_true_cv, y_pred_cv  = [], []

    print(f"  - Avaliando através de validação cruzada ({skf.n_splits}-folds)...")

    # Filtrando output/status por aparelho
    y_aparelho = y_cv[rotulo_aparelho]
    
    print("     -> Detalhes da amostragem:")
    print("     ---")
    for item in Counter(y_aparelho).items():
        print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_aparelho)*100,1)}%)" )
    print()
    

    print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
    for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X_cv, y_aparelho)), total=skf.n_splits):
        
        # Preparando lotes
        X_treino_cv, X_teste_cv = X_cv.iloc[idx_treino], X_cv.iloc[idx_teste]
        y_treino_cv, y_teste_cv = y_aparelho.iloc[idx_treino], y_aparelho.iloc[idx_teste]

        # Treinando modelo
        clf = clone(modelo)
        
        clf.fit(X_treino_cv, y_treino_cv)

        # Prevendo conjunto de dados
        y_hat = clf.predict(X_teste_cv)

        # Incrementando resultados
        resultados_modelo["appliance"].append(rotulo_aparelho)
        resultados_modelo["fold"].append(it+1)
        resultados_modelo["acc"].append( accuracy_score(y_teste_cv, y_hat) )
        resultados_modelo["f1"].append( f1_score(y_teste_cv, y_hat, average="macro") )
        resultados_modelo["auc"].append(roc_auc_score(y_teste_cv, y_hat) if np.unique(y_teste_cv).shape[0]>1 else 0.5)
        resultados_modelo["base"].append("cv")

        # Extendendo rotulos (analise global)
        y_true_cv.extend(y_teste_cv)
        y_pred_cv.extend(y_hat)

    print("      > Resultado:")
    print("        = Classification Report:")
    print()
    print(classification_report(y_true_cv, y_pred_cv))
    print("        = Confusion Matrix:")
    print()
    print(confusion_matrix(y_true_cv, y_pred_cv))
    print()
    print("**********************************************")
    print()

# Consolidando DataFrame
df_resultados = pd.DataFrame(resultados_modelo)

arquivo_resultados = os.path.join(caminho_dados_notebook, f"resultados_{dominio}_mlp.xlsx")
if os.path.isfile(arquivo_resultados): os.remove(arquivo_resultados)
df_resultados.to_excel(arquivo_resultados, index=False)
    
print("############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################")
display(df_resultados.groupby(["appliance","base"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))  

****************************************************************

* Aparelho `LC`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `1`: 12800 amostras (50.8%)
       - Classe `0`: 12400 amostras (49.2%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1.00      1.00     12800

    accuracy                           1.00     25200
   macro avg       1.00      1.00      1.00     25200
weighted avg       1.00      1.00      1.00     25200

        = Confusion Matrix:

[[12400     0]
 [    0 12800]]

  - Avaliando através de validação cruzada (5-folds)...
     -> Detalhes da amostragem:
     ---
        - Classe `1`: 32000 amostras (50.8%)
        - Classe `0`: 31000 amostras (49.2%)

     -> Avaliando modelo (CV - 5 folds)...



  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `LI`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31000
           1       1.00      1.00      1.00     32000

    accuracy                           1.00     63000
   macro avg       1.00      1.00      1.00     63000
weighted avg       1.00      1.00      1.00     63000

        = Confusion Matrix:

[[31000     0]
 [    0 32000]]

**********************************************

****************************************************************

* Aparelho `MO`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12400
           1       1.00      1

...

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.81      0.87     31000
           1       0.84      0.94      0.89     32000

    accuracy                           0.88     63000
   macro avg       0.88      0.88      0.88     63000
weighted avg       0.88      0.88      0.88     63000

        = Confusion Matrix:

[[25109  5891]
 [ 1845 30155]]

**********************************************

****************************************************************

* Aparelho `LF`...

  - Avaliando modelo através da base treino/validacao...
     -> Detalhes da amostragem (lote validacao):
     ---
       - Classe `0`: 12400 amostras (49.2%)
       - Classe `1`: 12800 amostras (50.8%)

     -> Treinando modelo...

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.87      0.87      0.87     12400
           1       0.87      0

  0%|          | 0/5 [00:00<?, ?it/s]

      > Resultado:
        = Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.90      0.90     31000
           1       0.91      0.90      0.90     32000

    accuracy                           0.90     63000
   macro avg       0.90      0.90      0.90     63000
weighted avg       0.90      0.90      0.90     63000

        = Confusion Matrix:

[[27998  3002]
 [ 3266 28734]]

**********************************************

############################## RESULTADO FINAL DO DOMINIO/MODELO ##############################


acc                                      f1  \
                            mean       std       max       min      mean   
appliance base                                                             
LC        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
LF        cv            0.900508  0.003033  0.904206  0.896032  0.900493   
          treino-teste  0.872024       NaN  0.872024  0.872024  0.871990   
LI        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
MO        cv            0.745317  0.001864  0.748254  0.743492  0.725425   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
MT        cv            1.000000  0.000000  1.000000  1.000000  1.000000   
          treino-teste  1.000000       NaN  1.000000  1.000000  1.000000   
PC        cv            0.877206  0.001313  0.878730  0.875714  0.876404   
          treino-teste  0.853214       NaN  0.853214  0.853214  0.852704   

                                                           auc            \
                             std       max       min      mean       std   
appliance base                                                             
LC        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
LF        cv            0.003038  0.904188  0.896004  0.900549  0.003056   
          treino-teste       NaN  0.871990  0.871990  0.871987       NaN   
LI        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
MO        cv            0.002426  0.729423  0.723475  0.741232  0.001905   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
MT        cv            0.000000  1.000000  1.000000  1.000000  0.000000   
          treino-teste       NaN  1.000000  1.000000  1.000000       NaN   
PC        cv            0.001292  0.877882  0.874976  0.876156  0.001278   
          treino-teste       NaN  0.852704  0.852704  0.852495       NaN   

                                            
                             max       min  
appliance base                              
LC        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
LF        cv            0.904214  0.895998  
          treino-teste  0.871987  0.871987  
LI        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
MO        cv            0.744254  0.739408  
          treino-teste  1.000000  1.000000  
MT        cv            1.000000  1.000000  
          treino-teste  1.000000  1.000000  
PC        cv            0.877629  0.874793  
          treino-teste  0.852495  0.852495

### Análise dos Resultados (1)

In [54]:
df_resultados_svm = pd.read_excel(os.path.join(caminho_dados_notebook, f"resultados_{dominio}_svm.xlsx"), engine='openpyxl')
df_resultados_svm["model"] = "SVM"

df_resultados_xgboost = pd.read_excel(os.path.join(caminho_dados_notebook, f"resultados_{dominio}_xgboost.xlsx"), engine='openpyxl')
df_resultados_xgboost["model"] = "XGBOOST"

df_resultados_mlp = pd.read_excel(os.path.join(caminho_dados_notebook, f"resultados_{dominio}_mlp.xlsx"), engine='openpyxl')
df_resultados_mlp["model"] = "MLP"

# df_resultados_elm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_elm.xlsx"))
# df_resultados_elm["model"] = "ELM"

df_analise = pd.concat([
    df_resultados_svm,
    df_resultados_xgboost,
    df_resultados_mlp, 
#     df_resultados_elm,  
])

print("* Análise por modelo:")
df_analise_modelo = df_analise.groupby(["model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("model")
display(df_analise_modelo)
df_analise_modelo.to_excel(os.path.join(caminho_dados_notebook, f"analise_{dominio}_modelos.xlsx"))

print()
print("* Análise por aparelho/modelo:")
df_analise_aparelho = df_analise.groupby(["appliance","model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, f"analise_{dominio}_aparelhos.xls"))

* Análise por modelo:


base       acc                                 f1            \
                           mean       std  max       min      mean       std   
model                                                                          
XGBOOST            cv  0.999989  0.000034  1.0  0.999841  0.999989  0.000034   
XGBOOST  treino-teste  0.989550  0.015440  1.0  0.960238  0.989548  0.015441   
MLP      treino-teste  0.954206  0.071192  1.0  0.853214  0.954116  0.071345   
SVM                cv  0.944894  0.096612  1.0  0.736190  0.944832  0.096722   
SVM      treino-teste  0.936224  0.106449  1.0  0.746032  0.936161  0.106510   
MLP                cv  0.920505  0.094627  1.0  0.743492  0.917054  0.101116   

                             auc                           
         max       min      mean       std  max       min  
model                                                      
XGBOOST  1.0  0.999841  0.999989  0.000034  1.0  0.999844  
XGBOOST  1.0  0.960235  0.989557  0.015411  1.0  0.960330  
MLP      1.0  0.852704  0.954080  0.071405  1.0  0.852495  
SVM      1.0  0.735730  0.944965  0.096713  1.0  0.735713  
SVM      1.0  0.745968  0.936380  0.106350  1.0  0.745968  
MLP      1.0  0.723475  0.919656  0.096015  1.0  0.739408


* Análise por aparelho/modelo:


acc                                \
                                    mean       std       max       min   
appliance model   base                                                   
LC        MLP     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          SVM     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          XGBOOST cv            0.999968  0.000043  1.000000  0.999921   
                  treino-teste  0.989762       NaN  0.989762  0.989762   
LF        MLP     cv            0.900508  0.003033  0.904206  0.896032   
                  treino-teste  0.872024       NaN  0.872024  0.872024   
          SVM     cv            0.740714  0.003310  0.744127  0.736190   
                  treino-teste  0.746032       NaN  0.746032  0.746032   
          XGBOOST cv            0.999968  0.000071  1.000000  0.999841   
                  treino-teste  0.960238       NaN  0.960238  0.960238   
LI        MLP     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          SVM     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          XGBOOST cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
MO        MLP     cv            0.745317  0.001864  0.748254  0.743492   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          SVM     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          XGBOOST cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  0.987302       NaN  0.987302  0.987302   
MT        MLP     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          SVM     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          XGBOOST cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
PC        MLP     cv            0.877206  0.001313  0.878730  0.875714   
                  treino-teste  0.853214       NaN  0.853214  0.853214   
          SVM     cv            0.928651  0.005950  0.935397  0.923413   
                  treino-teste  0.871310       NaN  0.871310  0.871310   
          XGBOOST cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   

                                      f1                                \
                                    mean       std       max       min   
appliance model   base                                                   
LC        MLP     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          SVM     cv            1.000000  0.000000  1.000000  1.000000   
                  treino-teste  1.000000       NaN  1.000000  1.000000   
          XGBOOST cv            0.999968  0.000043  1.000000  0.999921   
                  treino-teste  0.989761       NaN  0.989761  0.989761   
LF        MLP     cv            0.900493  0.003038  0.904188  0.896004   
                  treino-teste  0.871990       NaN  0.871990  0.871990   
          SVM     cv            0.740423  0.003435  0.743966  0.735730   
                  treino-teste  0.745968       NaN  0.745968  0.745968   
          XGBOOST cv            0.999968  0.000071  1.000000  0.999841   
                  treino-teste  0.960235       NaN  0.960235  0.960235   
LI        MLP     cv            1.000000

# Conclusões

...

# Fim.

In [55]:
%load_ext watermark

In [56]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Author: Diego Luiz Cavalca

Last updated: Mon Jan 24 2022 08:52:41Hora oficial do Brasil

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: 5e5bccaaf9e541e11be67706c7eb7d7b39a8be65

